In [102]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy.stats import mode
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn import tree
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import LabelEncoder


df=pd.read_csv('/resources/data/Big Mart Sales/train_bm1.csv')
df_test=pd.read_csv('/resources/data/Big Mart Sales/test_bm1.csv')

In [103]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [104]:
df_test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,Small,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,Small,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,Small,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [105]:
df.apply(lambda x: sum(x.isnull()),axis=0)

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [106]:
df_test.apply(lambda x: sum(x.isnull()),axis=0)

Item_Identifier                0
Item_Weight                  976
Item_Fat_Content               0
Item_Visibility                0
Item_Type                      0
Item_MRP                       0
Outlet_Identifier              0
Outlet_Establishment_Year      0
Outlet_Size                    0
Outlet_Location_Type           0
Outlet_Type                    0
dtype: int64

In [107]:
df['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [108]:
df['Outlet_Location_Type'].value_counts()

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64

In [109]:
df['Outlet_Type'].value_counts()

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

In [110]:
df['Outlet_Size'].value_counts()

Small     4798
Medium    2793
High       932
Name: Outlet_Size, dtype: int64

In [111]:
item_avg_weight = df.pivot_table(values='Item_Weight', index='Item_Identifier')
missing = df['Item_Weight'].isnull()
df.loc[missing,'Item_Weight'] = df.loc[missing,'Item_Identifier'].apply(lambda x: item_avg_weight[x])

In [112]:
item_avg_weight_test = df_test.pivot_table(values='Item_Weight', index='Item_Identifier')
missing_test = df_test['Item_Weight'].isnull()
df_test.loc[missing_test,'Item_Weight'] = df_test.loc[missing_test,'Item_Identifier'].apply(lambda x: item_avg_weight_test[x])

In [113]:
df.groupby(['Outlet_Type'])['Outlet_Size'].value_counts()

Outlet_Type        Outlet_Size
Grocery Store      Small          1083
Supermarket Type1  Small          3715
                   High            932
                   Medium          930
Supermarket Type2  Medium          928
Supermarket Type3  Medium          935
Name: Outlet_Size, dtype: int64

In [114]:
df_test.groupby(['Outlet_Type'])['Outlet_Size'].value_counts()

Outlet_Type        Outlet_Size
Grocery Store      Small           722
Supermarket Type1  Small          2476
                   High            621
                   Medium          620
Supermarket Type2  Medium          618
Supermarket Type3  Medium          624
Name: Outlet_Size, dtype: int64

In [115]:
df.dropna(inplace=True)


In [116]:
df_test['Item_Weight'].fillna(df_test['Item_Weight'].mean(),inplace=True)

In [117]:
df.Outlet_Type[df.Outlet_Type=='Supermarket Type3']='Supermarket Type2'
df.groupby(['Outlet_Type'])['Outlet_Size'].value_counts()

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Outlet_Type        Outlet_Size
Grocery Store      Small          1082
Supermarket Type1  Small          3715
                   High            932
                   Medium          930
Supermarket Type2  Medium         1860
Name: Outlet_Size, dtype: int64

In [118]:
df_test.Outlet_Type[df_test.Outlet_Type=='Supermarket Type3']='Supermarket Type2'
df_test.groupby(['Outlet_Type'])['Outlet_Size'].value_counts()

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Outlet_Type        Outlet_Size
Grocery Store      Small           722
Supermarket Type1  Small          2476
                   High            621
                   Medium          620
Supermarket Type2  Medium         1242
Name: Outlet_Size, dtype: int64

In [119]:
df.loc[df['Item_Visibility']==0.000000,'Item_Visibility']=df['Item_Visibility'].mean()

visibility_avg = df.pivot_table(values='Item_Visibility', index='Item_Identifier')
df['Item_Visibility_MeanRatio'] = df.apply(lambda x: x['Item_Visibility']/visibility_avg[x['Item_Identifier']], axis=1)

In [120]:
df_test.loc[df_test['Item_Visibility']==0.000000,'Item_Visibility']=df_test['Item_Visibility'].mean()

visibility_avg_test = df_test.pivot_table(values='Item_Visibility', index='Item_Identifier')
df_test['Item_Visibility_MeanRatio'] = df_test.apply(lambda x: x['Item_Visibility']/visibility_avg_test[x['Item_Identifier']], axis=1)

In [121]:
df['Item_Type_Combined'] = df['Item_Identifier'].apply(lambda x: x[0:2])

df['Item_Type_Combined'] = df['Item_Type_Combined'].map({'FD':'Food',
                                                             'NC':'Non-Consumable',
                                                             'DR':'Drinks'})
df['Item_Type_Combined'].value_counts()

Food              6121
Non-Consumable    1599
Drinks             799
Name: Item_Type_Combined, dtype: int64

In [122]:
df_test['Item_Type_Combined'] = df_test['Item_Identifier'].apply(lambda x: x[0:2])

df_test['Item_Type_Combined'] = df_test['Item_Type_Combined'].map({'FD':'Food',
                                                             'NC':'Non-Consumable',
                                                             'DR':'Drinks'})
df_test['Item_Type_Combined'].value_counts()

Food              4076
Non-Consumable    1087
Drinks             518
Name: Item_Type_Combined, dtype: int64

In [123]:
df['Outlet_Years'] = 2013 - df['Outlet_Establishment_Year']
df_test['Outlet_Years'] = 2013 - df_test['Outlet_Establishment_Year']

In [124]:
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                             'reg':'Regular',
                                                             'low fat':'Low Fat'})
df['Item_Fat_Content'].value_counts()

Low Fat    5516
Regular    3003
Name: Item_Fat_Content, dtype: int64

In [125]:
df_test['Item_Fat_Content'] = df_test['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                             'reg':'Regular',
                                                             'low fat':'Low Fat'})
df_test['Item_Fat_Content'].value_counts()

Low Fat    3668
Regular    2013
Name: Item_Fat_Content, dtype: int64

In [126]:
df.head(5)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Visibility_MeanRatio,Item_Type_Combined,Outlet_Years
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,0.922960,Food,14
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,1.003057,Drinks,4
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,0.831990,Food,14
3,FDX07,19.20,Regular,0.066112,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800,1.771921,Food,15
4,NCD19,8.93,Low Fat,0.066112,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,1.607074,Non-Consumable,26


In [127]:
df.loc[df['Item_Type_Combined']=="Non-Consumable",'Item_Fat_Content'] = "Non-Edible"
df['Item_Fat_Content'].value_counts()

Low Fat       3917
Regular       3003
Non-Edible    1599
Name: Item_Fat_Content, dtype: int64

In [128]:
df_test.loc[df_test['Item_Type_Combined']=="Non-Consumable",'Item_Fat_Content'] = "Non-Edible"
df_test['Item_Fat_Content'].value_counts()

Low Fat       2581
Regular       2013
Non-Edible    1087
Name: Item_Fat_Content, dtype: int64

In [129]:
le = LabelEncoder()
#New variable for outlet
df['Outlet'] = le.fit_transform(df['Outlet_Identifier'])
df_test['Outlet'] = le.fit_transform(df_test['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']
lf = LabelEncoder()
for i in var_mod:
    df[i] = lf.fit_transform(df[i])
    df_test[i] = lf.fit_transform(df_test[i])

In [130]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Visibility_MeanRatio,Item_Type_Combined,Outlet_Years,Outlet
0,FDA15,9.30,0,0.016047,Dairy,249.8092,OUT049,1999,1,0,1,3735.1380,0.922960,1,14,9
1,DRC01,5.92,2,0.019278,Soft Drinks,48.2692,OUT018,2009,1,2,2,443.4228,1.003057,0,4,3
2,FDN15,17.50,0,0.016760,Meat,141.6180,OUT049,1999,1,0,1,2097.2700,0.831990,1,14,9
3,FDX07,19.20,2,0.066112,Fruits and Vegetables,182.0950,OUT010,1998,2,2,0,732.3800,1.771921,1,15,0
4,NCD19,8.93,1,0.066112,Household,53.8614,OUT013,1987,0,2,1,994.7052,1.607074,2,26,1


In [131]:
df_test.head(2)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Visibility_MeanRatio,Item_Type_Combined,Outlet_Years,Outlet
0,FDW58,20.75,0,0.007565,Snack Foods,107.8622,OUT049,1999,1,0,1,0.453614,1,14,9
1,FDW14,8.30,2,0.038428,Dairy,87.3198,OUT017,2007,2,1,1,0.897571,1,6,2


In [132]:
x_train=df.drop(['Item_Type','Outlet_Establishment_Year','Item_Outlet_Sales','Item_Identifier','Outlet_Identifier'],axis=1)
y_train=df['Item_Outlet_Sales']

#x_train=x_train.astype(int)
#y_train=y_train.astype(int)

#y_train = np.asarray(df['Item_Outlet_Sales'], dtype="|S6")
x_test=df_test.drop(['Item_Type','Outlet_Establishment_Year','Item_Identifier','Outlet_Identifier'],axis=1)



In [133]:
#random_forest = RandomForestClassifier(n_estimators=20)

#random_forest.fit(x_train, y_train)

#pred = random_forest.predict(x_test)

#random_forest.score(x_train, y_train)

In [134]:
reg = linear_model.LinearRegression()
reg.fit (x_train, y_train)
reg.score(x_train, y_train)
pr=reg.predict(x_test)

In [135]:
submission = pd.DataFrame({
        "Item_Identifier": df_test["Item_Identifier"],
        "Outlet_Identifier":df_test["Outlet_Identifier"],
        "Item_Outlet_Sales": pr
    })
submission.to_csv('/resources/data/Big Mart Sales/Sample_Submission.csv',sep=',', index=False)